## Crawler

In this section, we'll crawl on [whoscored.com](https://www.whoscored.com/) to get data. This process will take some time (3 to 5 hours depending on the internet speed).

In [1]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

#Options to run browser on background.
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

"""
chromedriver.exe file should be in the same folder with this notebook.
Otherwise, path of chromedriver.exe should pe passed as an argument.
"""

driver = webdriver.Chrome(chrome_options=options)

#list of league page URLs to collect team urls
league_urls = ["https://www.whoscored.com/Regions/252/Tournaments/2/England-Premier-League",
               "https://www.whoscored.com/Regions/108/Tournaments/5/Italy-Serie-A",
               "https://www.whoscored.com/Regions/206/Tournaments/4/Spain-La-Liga",
               "https://www.whoscored.com/Regions/81/Tournaments/3/Germany-Bundesliga",
               "https://www.whoscored.com/Regions/74/Tournaments/22/France-Ligue-1",
               "https://www.whoscored.com/Regions/177/Tournaments/21/Portugal-Liga-NOS",
               "https://www.whoscored.com/Regions/155/Tournaments/13/Netherlands-Eredivisie",
               "https://www.whoscored.com/Regions/182/Tournaments/77/Russia-Premier-League",
               "https://www.whoscored.com/Regions/31/Tournaments/95/Brazil-Brasileir%C3%A3o",
               "https://www.whoscored.com/Regions/11/Tournaments/68/Argentina-Primera-Divisi%C3%B3n",
               "https://www.whoscored.com/Regions/225/Tournaments/17/Turkey-Super-Lig"]

The following is the function to get team URLs.

**Parameters**<br>
**driver:** Driver object.<br>
**league_url:** URL of the leage.<br>
**sleep time:** Time to wait for asynchronous loading before getting list of teams. Crawler may get list before data is being loaded and the result will be an empty dataframe. 5 seconds is ok.

**Returns:** A dataframe with list of teams. 

In [1]:
def get_team_urls(driver, league_url, sleep_time):
    
    #connect to league page
    driver.get(league_url)
    
    #wait for asynchronous loading
    time.sleep(sleep_time)
    
    #dataframe to hold team URLs
    teams_df = pd.DataFrame(columns=["team_id", "team_name", "team_url"])
    
    #get list of HTML elements for each team
    teams = driver.find_elements_by_css_selector(".standings > tr > .team")
    
    #get attributes for each team in the list
    for team in teams:
        team_dict = {
            "team_id": team.find_element_by_css_selector(".team-link").get_attribute("href").split("/")[4],
            "team_name": team.find_element_by_css_selector('.team-link').text,
            "team_url": team.find_element_by_css_selector('.team-link').get_attribute("href")
        }
    
        teams_df.loc[len(teams_df)] = team_dict
    
    return teams_df

In [3]:
#Function to replace "-" values in a dataframe as 0
def replace_dash(df):
    mapping = {'-': 0}
    replace_dict = {}
    
    for colum in df.columns:
        replace_dict[colum] = mapping
        
    return df.replace(replace_dict)

The following is the function to get summary stats of players in a team page.

**Parameters**<br>
**driver:** Driver object.<br>
**team_url:** URL of the team.<br>
**sleep time:** Time to wait for asynchronous loading before getting list of teams.

**Returns:** A dataframe with list of summary stats for each player in the team. 

In [4]:
def get_player_summary_stats(driver, team_url, sleep_time):
    
    #connect to the team page
    driver.get(team_url)
    
    #wait for asynchronous loading
    time.sleep(sleep_time)
    
    #dataframe to hold summary stats of players
    players_summary_df = pd.DataFrame(columns=["player_id", "player_name", "team", "country", "age",
                                               "position", "height_cm", "weight_kg", "first_eleven",
                                               "substituted_on", "minutes_played", "yellow_cards",
                                               "red_cards", "man_of_the_match", "avg_rating",
                                               "aerials_won"])
    
    #get list of HTML elements for each player
    players = driver.find_elements_by_css_selector("#player-table-statistics-body > tr")
    
    #get summary stats for each player in the list
    for player in players:
        
        #rarely, some unfamous players don't have profile page, which means they don't have player link and player id.
        #it throws an error. we'll pass those players. they will not included in data.
        try:
            
            #split number of games played as first eleven and number of games which a player substituted on
            apps = player.find_elements_by_css_selector("td")[5].text.strip().split("(")
            first_eleven = apps[0]
            if len(apps) > 1:
                substituted_on = apps[1].replace(")", "")
            else:
                substituted_on = 0
                
            #getting player attributes from HTML elements
            player_dict = {
                "player_id": player.find_element_by_css_selector(".player-link").get_attribute("href").split("/")[4],
                "player_name": player.find_element_by_css_selector(".player-link").text.strip(),
                "team": driver.find_element_by_css_selector(".team-header-name").text.strip(),
                "country": player.find_element_by_css_selector(".ui-icon").get_attribute("class").split(" ")[2].split("-")[1].strip(),
                "age": player.find_elements_by_css_selector(".pn > span")[0].text.strip(),
                "position": player.find_elements_by_css_selector(".pn > span")[1].text.replace(",", "", 1).strip(),
                "height_cm": player.find_elements_by_css_selector("td")[3].text.strip(),
                "weight_kg": player.find_elements_by_css_selector("td")[4].text.strip(),
                "first_eleven": first_eleven,
                "substituted_on": substituted_on,
                "minutes_played": player.find_element_by_css_selector(".minsPlayed").text.strip(),
                "yellow_cards": player.find_element_by_css_selector(".yellowCard").text.strip(),
                "red_cards": player.find_element_by_css_selector(".redCard").text.strip(),
                "man_of_the_match": player.find_element_by_css_selector(".manOfTheMatch").text.strip(),
                "avg_rating": player.find_element_by_css_selector(".rating").text.strip(),
                "aerials_won": player.find_element_by_css_selector(".aerialWonPerGame").text.strip()
            }

            players_summary_df.loc[len(players_summary_df)] = player_dict
        
        except:
            pass
    
    return replace_dash(players_summary_df)

The following is the function to get defensive stats of players in a team page.

**Parameters**<br>
**driver:** Driver object.<br>
**team_url:** URL of the team.<br>
**sleep time:** Time to wait for asynchronous loading before getting list of teams.

**Returns:** A dataframe with list of defensive stats for each player in the team. 

In [5]:
def get_player_defensive_stats(driver, team_url, sleep_time):
    
    #connect to the team page
    driver.get(team_url)
    
    #scroll down to click on defensive stats button without an error
    #otherwise driver may click to a disclaimer banner in front of the button
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    #click on defensive stats button to get defensive stats on the page
    driver.find_elements_by_css_selector("#team-squad-stats-options > li")[1].find_element_by_css_selector("a").click()
    
    #wait for asynchronous loading for defensive stats to be loaded
    time.sleep(sleep_time)
    
    #dataframe to hold defensive stats of players
    players_defensive_df = pd.DataFrame(columns=["player_id", "tackles", "interceptions", "fouls", "offsides_won",
                                                "clearances", "dribbled_past", "blocks", "own_goals"])
    
    #get defensive stats for each player in the list
    players = driver.find_elements_by_css_selector("#team-squad-stats-defensive #player-table-statistics-body > tr")
    
    for player in players:
        
        #we'll pass players without a profile link to avoid error
        try:
            
            #getting player attributes from HTML elements
            player_dict = {
                "player_id": player.find_element_by_css_selector(".player-link").get_attribute("href").split("/")[4],
                "tackles": player.find_element_by_css_selector(".tacklePerGame").text.strip(),
                "interceptions": player.find_element_by_css_selector(".interceptionPerGame").text.strip(),
                "fouls": player.find_element_by_css_selector(".foulsPerGame").text.strip(),
                "offsides_won": player.find_element_by_css_selector(".offsideWonPerGame").text.strip(),
                "clearances": player.find_element_by_css_selector(".clearancePerGame").text.strip(),
                "dribbled_past": player.find_element_by_css_selector(".wasDribbledPerGame").text.strip(),
                "blocks": player.find_element_by_css_selector(".outfielderBlockPerGame").text.strip(),
                "own_goals": player.find_element_by_css_selector(".goalOwn").text.strip(),
            }
            
            players_defensive_df.loc[len(players_defensive_df)] = player_dict
        
        except:
            pass
    
    return replace_dash(players_defensive_df)

The following is the function to get offensive stats of players in a team page.

**Parameters**<br>
**driver:** Driver object.<br>
**team_url:** URL of the team.<br>
**sleep time:** Time to wait for asynchronous loading before getting list of teams.

**Returns:** A dataframe with list of offensive stats for each player in the team. 

In [6]:
def get_player_offensive_stats(driver, team_url, sleep_time):
    
    #connect to the team page
    driver.get(team_url)
    
    #scroll down to click on offensive stats button without an error
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    #click on offensive stats button to get offensive stats on the page
    driver.find_elements_by_css_selector("#team-squad-stats-options > li")[2].find_element_by_css_selector("a").click()
    
    #wait for asynchronous loading for offensive stats to be loaded
    time.sleep(sleep_time)
    
    #dataframe to hold offensive stats of players
    players_offensive_df = pd.DataFrame(columns=["player_id", "goals", "assists", "shots", "key_passes", "dribbles",
                                                "fouled", "offsides", "dispossessed", "bad_controls"])
    
    #get offensive stats for each player in the list
    players = driver.find_elements_by_css_selector("#team-squad-stats-offensive #player-table-statistics-body > tr")
    
    for player in players:
        
        #we'll pass players without a profile link to avoid error
        try:
            
            #getting player attributes from HTML elements
            player_dict = {
                "player_id": player.find_element_by_css_selector(".player-link").get_attribute("href").split("/")[4],
                "goals": player.find_element_by_css_selector(".goal").text.strip(),
                "assists": player.find_element_by_css_selector(".assistTotal").text.strip(),
                "shots": player.find_element_by_css_selector(".shotsPerGame").text.strip(),
                "key_passes": player.find_element_by_css_selector(".keyPassPerGame").text.strip(),
                "dribbles": player.find_element_by_css_selector(".dribbleWonPerGame").text.strip(),
                "fouled": player.find_element_by_css_selector(".foulGivenPerGame").text.strip(),
                "offsides": player.find_element_by_css_selector(".offsideGivenPerGame").text.strip(),
                "dispossessed": player.find_element_by_css_selector(".dispossessedPerGame").text.strip(),
                "bad_controls": player.find_element_by_css_selector(".turnoverPerGame").text.strip()
            }
            
            players_offensive_df.loc[len(players_offensive_df)] = player_dict
            
        except:
            pass
    
    return replace_dash(players_offensive_df)

The following is the function to get passing stats of players in a team page.

**Parameters**<br>
**driver:** Driver object.<br>
**team_url:** URL of the team.<br>
**sleep time:** Time to wait for asynchronous loading before getting list of teams.

**Returns:** A dataframe with list of passing stats for each player in the team. 

In [7]:
def get_player_passing_stats(driver, team_url, sleep_time):
    
    #connect to the team page
    driver.get(team_url)
    
    #scroll down to click on passing stats button without an error
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    #click on passing stats button to get passing stats on the page
    driver.find_elements_by_css_selector("#team-squad-stats-options > li")[3].find_element_by_css_selector("a").click()
    
    #wait for asynchronous loading for passing stats to be loaded
    time.sleep(sleep_time)
    
    #dataframe to hold passing stats of players
    players_passing_df = pd.DataFrame(columns=["player_id", "passes", "pass_accuracy", "crosses", "long_balls",
                                              "through_balls"])
    
    #get passing stats for each player in the list
    players = driver.find_elements_by_css_selector("#team-squad-stats-passing #player-table-statistics-body > tr")
    
    for player in players:
        
        #we'll pass players without a profile link to avoid error
        try:
            
            #getting player attributes from HTML elements
            player_dict = {
                "player_id": player.find_element_by_css_selector(".player-link").get_attribute("href").split("/")[4],
                "passes": player.find_element_by_css_selector(".totalPassesPerGame").text.strip(),
                "pass_accuracy": player.find_element_by_css_selector(".passSuccess").text.strip(),
                "crosses": player.find_element_by_css_selector(".accurateCrossesPerGame").text.strip(),
                "long_balls": player.find_element_by_css_selector(".accurateLongPassPerGame").text.strip(),
                "through_balls": player.find_element_by_css_selector(".accurateThroughBallPerGame").text.strip()
            }
            
            players_passing_df.loc[len(players_passing_df)] = player_dict
        
        except:
            pass
    
    return replace_dash(players_passing_df)

Next and the last, we'll collect all player data into a single dataframe with using all functions defined above. 

In [8]:
#dataframe to hold all player data
player_stats_df = pd.DataFrame(columns=["player_id", "player_name", "team", "country", "age", "position", "height_cm", "weight_kg",
                                        "first_eleven", "substituted_on", "minutes_played", "yellow_cards", "red_cards",
                                        "man_of_the_match", "avg_rating", "aerials_won", "tackles", "interceptions", "fouls",
                                        "offsides_won", "clearances", "dribbled_past", "blocks", "own_goals", "goals",
                                        "assists", "shots", "key_passes", "dribbles", "fouled", "offsides", "dispossessed",
                                        "bad_controls", "passes", "pass_accuracy", "crosses", "long_balls", "through_balls"])

#loop through leagues
for league in league_urls:
    
    #get team URLs of the current league
    team_urls = get_team_urls(driver, league, 2)
    
    #loop through teams
    for team in team_urls["team_url"]:
        
        #get summary, defensive, offensive and passing stats for the current team
        summary_df = get_player_summary_stats(driver, team, 2)
        defensive_df = get_player_defensive_stats(driver, team, 2)
        offensive_df = get_player_offensive_stats(driver, team, 2)
        passing_df = get_player_passing_stats(driver, team, 2)
        
        #merge stats of the players in the current team into a single dataframe
        df_1 = summary_df.merge(defensive_df, on="player_id")
        df_2 = df_1.merge(offensive_df, on="player_id")
        df_3 = df_2.merge(passing_df, on="player_id")
        
        #add stats of the players for the current team into the main dataframe
        player_stats_df = player_stats_df.append(df_3)

#close the webdriver since its job has been finished
driver.close()

#correct indexes
player_stats_df.reset_index(drop=True, inplace=True)

#save dataframe into a csv file
player_stats_df.to_csv("data/player_stats.csv", encoding="utf-8")